In [ ]:
import pandas as pd
import numpy as np
from string import punctuation
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv('../input/squad-translated-to-tamil-for-chaii/squad_translated_tamil.csv')
hindi_stopwords=pd.read_csv('../input/hindi-stopwords/stopwords.txt', delimiter='\n', header=None)[0].values
tamil_stopwords=pd.read_csv('../input/tamilstopwords/tamil-stop-words.txt', delimiter='\n', header=None)[0].values

In [ ]:
train_answers=train.answer_text.values

In [ ]:
train.head()

In [ ]:
train_answers.shape

In [ ]:
len_ans=[len(ans) for ans in train_answers]

In [ ]:
np.mean(len_ans)

In [ ]:
#no of unique lengths of answer_text
ulen=list(set(len_ans))

In [ ]:
plt.figure(figsize=(10,10))
plt.bar(ulen, [len_ans.count(x) for x in ulen], width=10, color='orange', linewidth=0.5, edgecolor='black')
plt.xlabel('Length of answers')
plt.ylabel('Count')
plt.title('Count vs Length of answers')
plt.show()

In [ ]:
hcount=0
tcount=0
for ans in train_answers:
    tokens=ans.split()
    for token in tokens:
        if token in hindi_stopwords:
            hcount+=1
            break
        elif token in tamil_stopwords:
            tcount+=1
            break

In [ ]:
plt.figure(figsize=(5,5))
plt.bar(['Hindi', 'Tamil'],[hcount, tcount], width=0.5, color='orange', linewidth=0.5, edgecolor='black')
plt.xlabel('Language')
plt.ylabel('Count')
plt.title('No of answers containing at least one stopword')
plt.show()

In [ ]:
lengths=[]
for ans in train_answers:
    tokens=ans.split()
    for token in tokens:
        if token in hindi_stopwords or token in tamil_stopwords:
            lengths.append(len(ans))

In [ ]:
tamil_stopwords

In [ ]:
hsw_count=0
tsw_count=0
for ans in train_answers:
    tokens=ans.split()
    for token in tokens:
        if token in hindi_stopwords:
            hsw_count+=1
        elif token in tamil_stopwords:
            tsw_count+=1
            

In [ ]:
plt.figure(figsize=(5,5))
plt.bar(['Hindi', 'Tamil'],[hsw_count, tsw_count], width=0.5, color='orange', linewidth=0.5, edgecolor='black')
plt.xlabel('Language')
plt.ylabel('Count')
plt.title('No of stopwords in answers')
plt.show()

In [ ]:
# counting multiple occurances as well as dropping multiple occurences and removing unnecessary spaces
counts=[-1]*len(train)
context_df=[]
for i in range(len(train)):
    new_context=[]
    context=train.context.values[i]
    ans=train.answer_text.values[i]
    words_context=context.split()
    c=0
    flag=-1
    for word in words_context:
        if word==ans:
            if flag!=1:
                flag=1
                new_context.append(word)
            else:
                counts[i]=0
        else:
            new_context.append(word)
    context_df.append(' '.join(new_context))
    

In [ ]:
gr=counts.count(0)
ls=counts.count(-1)
plt.figure(figsize=(5,5))
plt.bar(['1', '>1'],[ls, gr], width=0.5, color='orange', linewidth=0.5, edgecolor='black')
plt.xlabel('Occurance of ans in the context')
plt.ylabel('Count')
plt.title('Multiple occurance of answer in the context')
plt.show()

In [ ]:
#Dropping answers with lengths>30
lengths=[len(ans) for ans in train.answer_text]
train['lengths']=lengths
train['context']=context_df
train.drop(train[train['lengths'] >30].index, inplace = True)

In [ ]:
train.shape

In [ ]:
train=train.reset_index(drop=True)

In [ ]:
# adjusting the answer_start
cz=['*']*len(train)
for i in range(len(train)):
    ans=train.answer_text[i]
    context=train.context[i]
    for c in range(len(context)):
        if context[c:c+len(ans)]==ans:
            cz[i]=c
            break
            

In [ ]:
for i in range(len(cz)):
    if cz[i]=='*':
        print(train.answer_text[i], train.answer_start[i], (train.context[i])[train.answer_start[i]:train.lengths[i]])

In [ ]:
train['answer_start']=cz

In [ ]:
train.drop(train[train['answer_start'] =='*'].index, inplace = True)
train=train.reset_index(drop=True)

In [ ]:
train

In [ ]:
train.context[3306][103:103+18]

In [ ]:
train=train.reset_index(drop=True)

In [ ]:
train.head()

In [ ]:
train.to_csv('Tamil_cleaned_Chaii.csv', index=False)